In [ ]:
import os
os.chdir('../../..')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
ll examples/

In [ ]:
from examples.two_tank_system.dataset import TwoTankImgDataSet
from torch.utils.data import DataLoader

In [ ]:
dataset = TwoTankImgDataSet(debug=True)

In [ ]:
batch_size = 1000
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=24)

In [ ]:
from sindy_autoencoder_cps.lightning_module import SINDyAutoencoder
import yaml
# MODEL_VERSION = 'freq_and_phase'
MODEL_VERSION = 'version_47'

# Cheating run: verion 29
# no cheating but not ideal: 32

hparams_path = f'./lightning_logs/{MODEL_VERSION}/hparams.yaml'
with open(hparams_path, 'r') as stream:
        hparam_dct = yaml.safe_load(stream)
ckpt_file_name = os.listdir(f'./lightning_logs/{MODEL_VERSION}/checkpoints/')[-1]
ckpt_file_path = f'./lightning_logs/{MODEL_VERSION}/checkpoints/{ckpt_file_name}'
model = SINDyAutoencoder.load_from_checkpoint(ckpt_file_path)
model

In [ ]:
batches = iter(dataloader)

In [ ]:
x, xdot, z_real, z_dot_real, idxs = batches.next()
print(f'shape x: {x.shape},   shape xdot: {xdot.shape}')

In [ ]:
import numpy as np
x_2_norm = np.array([v[0]**2+v[1]**2+v[2]**2 for v in x])
x_2_norm_sum = x_2_norm.sum()
x_2_norm_sum

In [ ]:
xdot_2_norm = np.array([v[0]**2+v[1]**2+v[2]**2 for v in xdot])
xdot_2_norm_sum = xdot_2_norm.sum()
xdot_2_norm_sum

In [ ]:
x_2_norm_sum / xdot_2_norm_sum

In [ ]:
1e3

In [ ]:
x_hat, xdot_hat, z, zdot, zdot_hat = model.cuda()(x.cuda(), xdot.cuda())

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np


fig = make_subplots(rows=2, cols=6, shared_xaxes=False)
for i in range(6):
    fig.add_trace(go.Scatter(x=x[:, i].cpu().detach().numpy(),
                       y=x_hat[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'x_hat{i} over x{i}',
#                              marker_color='blue',
                             opacity=.7), row=1, col=1+i)
    fig.add_trace(go.Scatter(x=xdot[:, i].cpu().detach().numpy(),
                       y=xdot_hat[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'xdot_hat{i} over xdot{i}',
#                              marker_color='blue',
                             opacity=.7), row=2, col=1+i)
fig.show()


In [ ]:

fig = make_subplots(rows=3, cols=2, shared_xaxes=False)
for j in range(2):
    fig.add_trace(go.Scatter(x=zdot[:, j].cpu().detach().numpy(),
                       y=zdot_hat[:, j].cpu().detach().numpy(),
                      mode='markers', name=f'zdot_hat{j} over zdot{j}',
#                              marker_color='blue',
                             opacity=.7), row=1, col=1+j)
    fig.add_trace(go.Scatter(x=z[:, j].cpu().detach().numpy(),
                       y=z_real[:, j].cpu().detach().numpy(),
                      mode='markers', name=f'z{j} over zreal{j}',
#                              marker_color='blue',
                             opacity=.7), row=2, col=1+j)
    fig.add_trace(go.Scatter(x=zdot[:, j].cpu().detach().numpy(),
                       y=z_dot_real[:, j].cpu().detach().numpy(),
                      mode='markers', name=f'zdot{j} over zdot_real{j}',
#                              marker_color='blue',
                             opacity=.7), row=3, col=1+j)
fig.show()


In [ ]:
import pandas as pd
XI = model.XI.cpu().detach().numpy()
XI.max()
df_XI = pd.DataFrame(XI, columns=['z0_dot_hat', 'z1_dot_hat'])
df_XI.index=model.SINDyLibrary.feature_names

In [ ]:
import plotly.express as px
print(model.SINDyLibrary.feature_names)
px.imshow(df_XI.abs())

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

fig = make_subplots(rows=2, cols=3, shared_xaxes=True)

for i in range(2):
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=z[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'z{i}',
                             marker_color='blue', opacity=.5), row=1, col=1+i,
                    )
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=z_real[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'zreal{i}',
                             marker_color='green', opacity=.5), row=1, col=1+i,
                    )
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=zdot_hat[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'zdot_hat{i}',
                            marker_color='blue', opacity=.4), row=2, col=1+i)
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=zdot[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'zdot{i}',
                             marker_color='green', opacity=.5), row=2, col=1+i)
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=z_dot_real[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'zdot_real{i}',
                             marker_color='purple', opacity=.5), row=2, col=1+i)



fig.update_layout(height=600, width=1200, title_text="Side By Side Subplots")
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

fig = make_subplots(rows=2, cols=4, shared_xaxes=True)


for i in range(4):
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=x[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'x{i}',
                             marker_color='purple', opacity=.5), row=1, col=1+i)
for i in range(4):
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=xdot[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'xdot{i}',
                             marker_color='red', opacity=.5), row=2, col=1+i)
for i in range(4):
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=xdot_hat[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'xdot_hat{i}',
                             marker_color='yellow', opacity=.5), row=2, col=1+i)



fig.update_layout(height=600, width=1200, title_text="Side By Side Subplots")
fig.show()

In [ ]:
import examples.two_tank_system.constants as const
# z_real = dataset.df.iloc[idxs][const.Z_COL_NAMES]
fig = make_subplots(rows=3, cols=3, shared_xaxes=True)

for i in range(2):
    for j in range(2):
        fig.add_trace(go.Scatter(y=z.detach().cpu().numpy()[:, i],
                           x=z_real.detach().cpu().numpy()[:, j],
                          mode='markers', name=f'z{i} over z_real{j}'), row=1+j, col=1+i)


fig.update_layout(height=600, width=1200, title_text="Real hidden state vs latent activations")
fig.show()

In [ ]:
zdot_real = dataset.df.iloc[idxs][const.Z_DOT_COL_NAMES]
fig = make_subplots(rows=3, cols=3, shared_xaxes=True)

for i in range(2):
    for j in range(2):
        fig.add_trace(go.Scatter(y=zdot.detach().cpu().numpy()[:, i],
                           x=zdot_real.values[:, j],
                          mode='markers', name=f'zdot{i} over zdot_real{j}'), row=1+j, col=1+i)


fig.update_layout(height=600, width=1200, title_text="Real hidden states derivatives vs predicted derivatives")
fig.show()

## prediction possible?

In [ ]:
# take sample from observation space
x_sample = x[0,:]
x_sample

In [ ]:
# get z coordinates
z_sample  = model.phi_x(x_sample.cuda())
z_sample

In [ ]:
# get model
thres = 0.005
XI = model.XI.detach().cpu().numpy()
indices_var = np.where(np.abs(XI)>thres)[1]
indices_expr = np.where(np.abs(XI)>thres)[0]
indices = list(zip(indices_expr, indices_var))
values = XI[np.abs(XI) > thres]
feature_names = model.SINDyLibrary.feature_names
z0_dot_expr = '+'.join([f'({XI[i]}*{feature_names[i[0]]})' for i in indices if i[1] == 0])
z1_dot_expr = '+'.join([f'({XI[i]}*{feature_names[i[0]]})' for i in indices if i[1] == 1])
z2_dot_expr = '+'.join([f'({XI[i]}*{feature_names[i[0]]})' for i in indices if i[1] == 2])
z0_dot_expr = 'z0_dot = '+z0_dot_expr
z1_dot_expr = 'z1_dot = '+z1_dot_expr
z2_dot_expr = 'z2_dot = '+z2_dot_expr
print(z0_dot_expr)
print(z1_dot_expr)
print(z2_dot_expr)

In [ ]:
# build ode model (manual!!!)
def ode_model(z, t):
    z0 = z[0]
    z1 = z[1]
    z0_dot =  0
    z1_dot =  (-0.006624102592468262*z0)+(0.08263646811246872*np.cos(z1))+(-0.006367685738950968*z0*z0*z0)+(-0.0212117787450552*z1*z1*z1)
    return z0_dot, z1_dot

In [ ]:
from scipy.integrate import odeint
import numpy as np
t = np.linspace(0, 20, 50)
z_init = z_sample
z_pred = odeint(ode_model, z_sample.detach().cpu().numpy(),
                t).astype(np.float32)

In [ ]:
import torch
x_pred = model.psi_z(torch.tensor(z_pred).cuda()).detach().cpu().numpy()
x_pred.shape

In [ ]:
const.X_COL_NAMES

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
# signal 1
for i in range(5):
    fig.add_trace(go.Scatter(x=list(range(50)), y=x_pred[:, i+1],
                  mode="lines", opacity=1, name=f'pred{i}'), row=1, col=1)
    fig.add_trace(go.Scatter(x=list(range(50)), y=x[:, i+1],
                  mode="lines", opacity=1, name=f'true{i}'), row=1, col=1)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
# signal 1
for i in range(3):
    fig.add_trace(go.Scatter(x=list(range(50)), y=x[:, i+1],
                  mode="lines", opacity=1), row=1, col=1)
fig.show()